## 1. Getting data from One Drive

In [2]:
from O365 import Account, FileSystemTokenBackend


In [8]:
ona = OneDriveAPI(tenant_id="e80d8e75-52b9-4839-a358-87abb93b3567",
                  client_id="ppeng60",
                  client_secret="Ag4das315Lsq01")

## 2. Joining different data sets

In [11]:
!ls ../../data/macro

China EU    US


In [13]:
import pandas as pd


In [35]:
import pandas as pd
import numpy as np
from pathlib import Path
import datetime as dt

global_macro = pd.read_pickle("../../data/macro/macro_data.pkl")
global_macro = global_macro.swaplevel(1, 0, axis=1).T.sort_index().T

def to_datetime(dt_str: str, format: str="%Y-%m-%d"):
    return dt.datetime.strptime(dt_str, format)


fred_data = []
for dir in Path("../../data/macro").iterdir():
    if dir.name in ["China", "EU", "US"]:
        cname = dir.name
        print(cname)
        for subd in dir.iterdir():
            if subd.suffix == ".csv":
                print(subd.name)
                df = pd.read_csv(subd.as_posix(), index_col=0, header=0)
                try:
                    df.index = df.index.to_series().apply(lambda x: pd.Period(to_datetime(x), freq="M"))
                except ValueError as e:
                    df.index = df.index.to_series().apply(lambda x: pd.Period(to_datetime(x, format="%Y-%m"), freq="M"))
                
                try:
                    df.index.name = "date"
                    df = df.reset_index().groupby(df.index.name).mean(numeric_only=True)
                except Exception as e:
                    print(e)
                    print(df)

                df.columns = pd.MultiIndex.from_product([[cname], df.columns])
                fred_data.append(df)

US
PAYEMS.csv
DGS10.csv
test_cpi.csv
fred_res.csv
MacroIndicatorsConcat.csv
T10Y2Y.csv
fred_res_gdp_research.csv
INDPRO.csv
Real GDP.csv
fred_res_fci_fin_research.csv
UNRATE.csv
ICSA.csv
test_fsi.csv
fred_res_cpi.csv
macro_data_us.csv
fred_res_fci_econ_research.csv
test_gdp.csv
fred_res_emp.csv
China
wind_fci_data.csv
wind_cpi_data.csv
wind_data.csv
EU
fred_res_cpi_eu.csv


In [36]:
fred_concat = pd.concat(fred_data, axis=1).sort_index()
fred_concat.columns.names = ["Country", "data_field"]
global_macro.index.name = "date"
global_macro.columns.names = ["Country", "data_field"]
consolidated = fred_concat.join(global_macro, how="outer").sort_index().T.sort_index().T

In [55]:
import plotly.express as px
import plotly.graph_objects as go

In [61]:
differenced = consolidated.US.pct_change(periods=-1)
corr_mat = differenced.corr(method="spearman")
plotly_obj = px.imshow(corr_mat, range_color=[-0.5, 0.5])
fig = go.Figure(plotly_obj)
fig.update_layout(title="correlation matrix")

In [63]:
consolidated.to_pickle("../../data/macro/consolidated_macro_data.pkl")